In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import contextily as ctx
from sodapy import Socrata
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from geopy.distance import geodesic


gdf = pd.read_csv('data/la_crashes_16_19.csv')


gdf = gdf.drop_duplicates()
gdf = gpd.GeoDataFrame(gdf, 
                       crs='epsg:4326',
                       geometry=gpd.points_from_xy(gdf.POINT_X, gdf.POINT_Y))


gdf = gdf[~gdf.geometry.is_empty]
gdf = gdf[gdf.geometry.is_valid]

center_coords = (34.088521, -118.306429)


gdf['distance_to_center'] = gdf.geometry.apply(lambda x: geodesic(center_coords, (x.y, x.x)).miles)
one_mile_radius = .1
gdf['distance_to_center'] = gdf.geometry.apply(lambda x: geodesic(center_coords, (x.y, x.x)).miles)
gdf = gdf[gdf['distance_to_center'] <= one_mile_radius]

collision_mapping = {
    'A': 'Head-On',
    'B': 'Sideswipe',
    'C': 'Rear End',
    'D': 'Broadside',
    'E': 'Hit Object',
    'F': 'Overturned',
    'G': 'Vehicle/Pedestrian',
    'H': 'Other'
}




gdf['TYPE_OF_COLLISION'] = gdf['TYPE_OF_COLLISION'].replace(collision_mapping)

mymap = folium.Map(location=center_coords, zoom_start=15)
mymap.get_root().html.add_child(folium.Element('2015_2022_Collisions.html'))
marker_cluster = MarkerCluster().add_to(mymap)

for index, row in gdf.iterrows():
    popup_text = f"Date: {row['COLLISION_DATE']}<br>CASE_ID: {row['CASE_ID']}<br>Collision Type: {row['TYPE_OF_COLLISION']}<br>NUMBER KILLED: {row['NUMBER_KILLED']}"
    popup = folium.Popup(popup_text, max_width=300)

    if row['TYPE_OF_COLLISION'] == 'Vehicle/Pedestrian':
        icon_color = 'red'
    elif row['NUMBER_KILLED'] == 1:
        icon_color = 'purple'
        print(f"CASE_ID: {row['CASE_ID']} has NUMBER_KILLED equal to 1")
    else:
        icon_color = 'blue'
    folium.Marker([row['geometry'].y, row['geometry'].x], popup=popup, icon=folium.Icon(color=icon_color)).add_to(marker_cluster)

#mymap.save('interactive_map_filtered.html')
mymap

/tmp/ipykernel_1231/2820971505.py:12: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  gdf = pd.read_csv('data/la_crashes_16_19.csv')
